### **0. Подготовка обучающей и тестовой выборки**

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
def evaluate_model(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name}: MAE = {mae:.2f}, MSE = {mse:.2f}, R² = {r2:.2f}")

In [3]:
# 1. Загрузка данных
boston = fetch_openml(name='boston', as_frame=True)
X = boston.data
y = boston.target

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
print("Пропуски в данных:", X.isnull().sum().sum())

Пропуски в данных: 0


In [5]:
# Масштабирование
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### **1. Обучение моделей**

#### **1.1 Стекинг**

In [7]:
base_models = [
    ('ridge', Ridge()),
    ('tree', DecisionTreeRegressor(random_state=42))
]

stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge()
)

stacking_model.fit(X_train, y_train)
y_pred_stack = stacking_model.predict(X_test)
evaluate_model("Stacking", y_test, y_pred_stack)

Stacking: MAE = 2.42, MSE = 12.36, R² = 0.83


#### **1.2 Многослойный перцептрон**

Как бэггинг, только случайна не только обучающая подвыборка, но и подмножество обучающих признаков

In [8]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
evaluate_model("MLP", y_test, y_pred_mlp)

MLP: MAE = 2.32, MSE = 13.18, R² = 0.82


#### **1.3 COMBI (линейный)**

метод перебора комбинаций математических операций и переменных

In [9]:
import gmdh
combi_model = gmdh.Combi()
combi_model.fit(X_train, y_train)
y_pred_combi = combi_model.predict(X_test)
evaluate_model("COMBI", y_test, y_pred_combi)

COMBI: MAE = 3.20, MSE = 25.15, R² = 0.66


#### **1.4 MIA (нелинейный)**

итеративный алгоритм, который строит модель по слоям, усложняя её на каждом шаге.

In [10]:
mia_model = gmdh.Mia()
mia_model.fit(X_train, y_train)
y_pred_mia = mia_model.predict(X_test)
evaluate_model("MIA", y_test, y_pred_mia)

MIA: MAE = 2.82, MSE = 19.45, R² = 0.73


### **2. Сравнение моделей**

1) MAE (Mean Absolute Error) - средняя абсолютная ошибка
2) MSE (Mean Squared Error) - средняя квадратичная ошибка
3)R^2 (R-squared) - доля объяснённой дисперсии

In [11]:
print("\nОценка качества моделей:")
evaluate_model("Stacking", y_test, y_pred_stack)
evaluate_model("MLP", y_test, y_pred_mlp)
evaluate_model("COMBI", y_test, y_pred_combi)
evaluate_model("MIA", y_test, y_pred_mia)


Оценка качества моделей:
Stacking: MAE = 2.42, MSE = 12.36, R² = 0.83
MLP: MAE = 2.32, MSE = 13.18, R² = 0.82
COMBI: MAE = 3.20, MSE = 25.15, R² = 0.66
MIA: MAE = 2.82, MSE = 19.45, R² = 0.73


In [2]:
!pip install numpy==1.24.4 scikit-learn==1.2.2 pandas==1.5.3 gmdh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.3 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26